In [4]:
from rdkit import Chem
import pandas as pd

In [5]:
filename = "/home/brennera/mystore/data/bind_db/full_bindingdb/BindingDB_All_3D.sdf"  # replace with actual path
supplier = Chem.ForwardSDMolSupplier(filename)

unique_pdb_ids = set()
batch_size = 5000
batch = []

for i, mol in enumerate(supplier):
    if mol is None:
        continue

    batch.append({'ROMol': mol, **mol.GetPropsAsDict()})

    if len(batch) == batch_size:
        df = pd.DataFrame(batch)

        pdb_id_lists = [
            [pdb.strip() for pdb in str(pdb_string).split(',') if pdb.strip()]
            if pd.notna(pdb_string) else []
            for pdb_string in df['PDB ID(s) of Target Chain']
        ]

        unique_pdb_ids.update(pdb_id for sublist in pdb_id_lists for pdb_id in sublist)

        batch = []

# Process remaining molecules
if batch:
    df = pd.DataFrame(batch)
    pdb_id_lists = [
        [pdb.strip() for pdb in str(pdb_string).split(',') if pdb.strip()]
        if pd.notna(pdb_string) else []
        for pdb_string in df['PDB ID(s) of Target Chain']
    ]
    unique_pdb_ids.update(pdb_id for sublist in pdb_id_lists for pdb_id in sublist)

# Final flat, sorted list of unique PDB IDs
flat_unique_pdb_ids = sorted(unique_pdb_ids)


OSError: File error: Bad input file /home/brennera/mystore/data/bind_db/full_bindin

In [ ]:
flat_unique_pdb_ids